In [1]:
"""
-------------------------------------------------------------------------------
HDR BATCH PROCESSOR
-------------------------------------------------------------------------------
A program to batch process HDR images<br>
Author: Brian Willis<br>
AI programming support: Gemini 3 pro<br>
Contact: Willis77019@gmail.com<br>
License:  MIT License (unrestricted: Use at your own risk)<br>

DESCRIPTION:
This script processes sets of photos to create High Dynamic Range (HDR) images
or Focus Stacks using Hugin's command-line tools. It supports .JPG and Nikon 
RAW (.NEF) files. The script employs robust error checking. 

If a set of bracketed photos cannot be aligned mathematically (due to extreme movement 
or lack of overlapping detail between exposures), the script will skip that set and 
provide a log detail explaining why, rather than crashing the entire batch.

PREREQUISITES:
1. Hugin Panorama Stitcher must be installed.
   - Point the script to the 'bin' directory containing "align_image_stack".
2. Python Libraries:
   - pip install rawpy imageio exifread

CONFIGURATION GUIDE:
- HDR Mode (Default): Keeps --exposure-weight=1.0 and --contrast-weight=0.0.
- Focus Stacking: Flip weights (--exposure-weight=0.0, --contrast-weight=1.0).
-------------------------------------------------------------------------------
"""

import os
import subprocess
import glob
import datetime
import shutil
import platform
import sys
import tkinter as tk
from tkinter import filedialog

# ================= USER CONFIGURATION =================

# 1. HUGIN PATH DETECTION
# Auto-detects OS to set the correct path and extension (.exe)
if platform.system() == "Windows":
    HUGIN_EXEC_EXT = ".exe"
    # Default Windows path (Change if you installed Hugin elsewhere)
    HUGIN_BIN_PATH = r"C:\Program Files\Hugin\bin" 
elif platform.system() == "Darwin": # Mac
    HUGIN_EXEC_EXT = ""
    HUGIN_BIN_PATH = "/Applications/Hugin/HuginTools"
else: # Linux
    HUGIN_EXEC_EXT = ""
    HUGIN_BIN_PATH = "/usr/bin"

# 2. GROUPING SETTINGS
# 'TIME': Groups photos by time bursts (BURST_THRESHOLD seconds apart).
# 'FIXED': Forces groups of a specific count (e.g. 3 photos per HDR).
GROUP_METHOD = 'FIXED' 
BURST_THRESHOLD = 2.0  # Seconds between bursts (for TIME mode)
FIXED_COUNT = 3        # Photos per set (for FIXED mode)

# 3. PROCESSING SETTINGS
# True = HDR (Exposure Blending). False = Focus Stacking (Contrast Blending).
DO_HDR_BLENDING = True 

# ======================================================

def get_timestamp(file_path):
    import exifread
    try:
        with open(file_path, 'rb') as f:
            tags = exifread.process_file(f, stop_tag='EXIF DateTimeOriginal')
            date_str = str(tags.get('EXIF DateTimeOriginal'))
            return datetime.datetime.strptime(date_str, '%Y:%m:%d %H:%M:%S')
    except:
        return datetime.datetime.fromtimestamp(os.path.getmtime(file_path))

def check_hugin_install():
    test_tool = os.path.join(HUGIN_BIN_PATH, f"align_image_stack{HUGIN_EXEC_EXT}")
    if not os.path.exists(test_tool):
        print("!!!" * 10)
        print(f"ERROR: Hugin tool not found at: {test_tool}")
        print("Please edit 'HUGIN_BIN_PATH' in the script configuration.")
        print("!!!" * 10)
        return False
    return True

def process_set(group, output_folder, set_index):
    # 1. Setup paths
    base_name = os.path.splitext(os.path.basename(group[0]))[0]
    output_filename = f"HDR_{base_name}.tif"
    output_path = os.path.join(output_folder, output_filename)
    
    print(f"  -> Processing set {set_index} ({len(group)} images)...")
    
    align_tool = os.path.join(HUGIN_BIN_PATH, f"align_image_stack{HUGIN_EXEC_EXT}")
    enfuse_tool = os.path.join(HUGIN_BIN_PATH, f"enfuse{HUGIN_EXEC_EXT}")
    
    # === STEP 1: DEVELOP RAWs TO TEMP TIFs ===
    # align_image_stack often fails on NEFs directly. We must feed it TIFs.
    temp_input_tifs = []
    
    try:
        import rawpy
        import imageio
        
        for raw_file in group:
            # Create a temp filename in the output folder
            fname = os.path.basename(raw_file)
            tif_name = os.path.join(output_folder, f"temp_dev_{fname}.tif")
            
            # Skip conversion if input is already TIF/JPG
            if raw_file.lower().endswith(('.tif', '.tiff', '.jpg', '.jpeg')):
                temp_input_tifs.append(raw_file)
                continue
                
            # Develop NEF to TIF
            # use_camera_wb=True: Keeps colors looking natural
            # no_auto_bright=True: CRITICAL. Preserves the exposure difference between brackets!
            with rawpy.imread(raw_file) as raw:
                rgb = raw.postprocess(use_camera_wb=True, no_auto_bright=True, bright=1.0, user_sat=None)
                imageio.imsave(tif_name, rgb)
                temp_input_tifs.append(tif_name)
                
    except Exception as e:
        print(f"    [ERROR] Failed to convert RAW: {e}")
        return

    # === STEP 2: ALIGN IMAGES ===
    # Now we feed the TIFs to align_image_stack
    cmd_align = [align_tool, "-m", "-a", "temp_aligned_", "-c", "24", "-v"] 
    cmd_align.extend([os.path.basename(f) for f in temp_input_tifs]) # Use filenames only
    
    try:
        # Run Hugin inside the output folder
        result = subprocess.run(
            cmd_align, 
            cwd=output_folder,
            capture_output=True,
            text=True
        )
        
        if result.returncode != 0:
            print("    [ERROR] Alignment failed.")
            print(f"    [HUGIN LOG] {result.stderr[:400]}") # Print more error info
            # Cleanup temp input tifs before returning
            for f in temp_input_tifs:
                if "temp_dev_" in f: os.remove(f)
            return

    except Exception as e:
        print(f"    [SYSTEM ERROR] {e}")
        return

    # Find the aligned TIFs created by Hugin
    aligned_files = sorted(glob.glob(os.path.join(output_folder, "temp_aligned_*.tif")))
    
    if not aligned_files:
        print("    [ERROR] No aligned files created (Silent Failure).")
        # Cleanup temp input tifs
        for f in temp_input_tifs:
            if "temp_dev_" in f: os.remove(f)
        return

    # === STEP 3: ENFUSE (BLEND) ===
    if DO_HDR_BLENDING:
        exp_w, cont_w = "1.0", "0.0"
    else:
        exp_w, cont_w = "0.0", "1.0"

    cmd_enfuse = [enfuse_tool, "--output", output_filename,
           f"--exposure-weight={exp_w}", 
           f"--saturation-weight=0.2", 
           f"--contrast-weight={cont_w}", 
           "--soft-mask"]
    cmd_enfuse.extend([os.path.basename(f) for f in aligned_files])
    
    subprocess.call(cmd_enfuse, cwd=output_folder)
    
    # === CLEANUP ===
    # Delete the "temp_dev" TIFs and the "temp_aligned" TIFs
    for f in aligned_files:
        try: os.remove(f)
        except: pass
        
    for f in temp_input_tifs:
        # Only delete files we created (containing "temp_dev_")
        if "temp_dev_" in f:
            try: os.remove(f)
            except: pass
        
    print(f"    [DONE] Saved to: {output_filename}")

def main():
    print("--- HDR BATCH PROCESSOR STARTING ---")
    if not check_hugin_install(): return

    # Windows Folder Picker
    root = tk.Tk()
    root.withdraw() # Hide the main window
    print("Please select your INPUT folder from the popup window...")
    INPUT_FOLDER = filedialog.askdirectory(title="Select Input Folder Containing Photos")
    
    if not INPUT_FOLDER:
        print("No folder selected. Exiting.")
        return

    OUTPUT_FOLDER = os.path.join(INPUT_FOLDER, "Processed_HDR")
    if not os.path.exists(OUTPUT_FOLDER): os.makedirs(OUTPUT_FOLDER)

    # Gather Files (Case-Insensitive to fix duplicate bugs)
    extensions = ['*.NEF', '*.nef', '*.JPG', '*.jpg', '*.TIF', '*.tif']
    raw_files = []
    for ext in extensions:
        raw_files.extend(glob.glob(os.path.join(INPUT_FOLDER, ext)))
    
    # Remove duplicates by normalizing to lowercase path
    unique_files = {}
    for f in raw_files:
        unique_files[os.path.abspath(f).lower()] = f
    files = sorted(list(unique_files.values()))

    if not files:
        print(f"No images found in {INPUT_FOLDER}")
        return

    # Grouping Logic
    groups = []
    if GROUP_METHOD == 'FIXED':
        for i in range(0, len(files), FIXED_COUNT):
            groups.append(files[i:i+FIXED_COUNT])
    else:
        # Time-based grouping
        files_with_time = [(f, get_timestamp(f)) for f in files]
        files_with_time.sort(key=lambda x: x[1])
        
        current_group = [files_with_time[0][0]]
        for i in range(1, len(files_with_time)):
            prev_time = files_with_time[i-1][1]
            curr_file, curr_time = files_with_time[i]
            
            if (curr_time - prev_time).total_seconds() < BURST_THRESHOLD:
                current_group.append(curr_file)
            else:
                groups.append(current_group)
                current_group = [curr_file]
        groups.append(current_group)

    print(f"Found {len(groups)} sets to process.")
    
    for i, group in enumerate(groups):
        if len(group) < 2:
            print(f"Skipping set {i+1} (Single image, need at least 2).")
            continue
        process_set(group, OUTPUT_FOLDER, i+1)

    print("--- ALL DONE ---")

if __name__ == "__main__":
    main()

--- HDR BATCH PROCESSOR STARTING ---
Please select your INPUT folder from the popup window...
No images found in C:/Users/Willi/Desktop/HDR TST/Stop
